In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

# Load portfolio volatility from previous step
df = pd.read_csv("stock_data.csv", index_col="date", parse_dates=True)

# Compute daily percentage returns
returns = df.pct_change().dropna()

# Calculate portfolio volatility (rolling standard deviation over 5 days)
df["volatility"] = returns.std(axis=1).rolling(window=5).mean()

# Drop NaN values
df.dropna(inplace=True)

# Normalize the volatility for LSTM (0 to 1 scaling)
scaler = MinMaxScaler()
df["volatility_scaled"] = scaler.fit_transform(df[["volatility"]])

# Prepare data
volatility_series = df["volatility_scaled"].values.reshape(-1, 1)

# Plot volatility over time
plt.figure(figsize=(10, 5))
plt.plot(df.index, df["volatility_scaled"], label="Scaled Volatility (Risk)")
plt.xlabel("Date")
plt.ylabel("Volatility (Scaled)")
plt.legend()
plt.show()


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Prepare input sequences
seq_length = 10  # Using last 10 days to predict the next day

X, y = [], []
for i in range(len(volatility_series) - seq_length):
    X.append(volatility_series[i : i + seq_length])
    y.append(volatility_series[i + seq_length])

X, y = np.array(X), np.array(y)

# Split into training and testing data (80-20 split)
split = int(len(X) * 0.8)
X_train, X_test, y_train, y_test = X[:split], X[split:], y[:split], y[split:]

# Build LSTM Model
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(seq_length, 1)),
    LSTM(50, return_sequences=False),
    Dense(25, activation="relu"),
    Dense(1)
])

# Compile the model
model.compile(optimizer="adam", loss="mse")

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=8, validation_data=(X_test, y_test))


In [ ]:
# Predict future volatility movement
predictions = model.predict(X_test)

# Inverse transform to get actual volatility values
predicted_volatility = scaler.inverse_transform(predictions)

# Plot actual vs predicted seeker movement
plt.figure(figsize=(10, 5))
plt.plot(df.index[-len(y_test):], scaler.inverse_transform(y_test.reshape(-1, 1)), label="Actual Seeker Movement")
plt.plot(df.index[-len(y_test):], predicted_volatility, label="Predicted Seeker Path", linestyle="dashed")
plt.xlabel("Date")
plt.ylabel("Seeker Movement (Risk-Based)")
plt.legend()
plt.show()
